In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM
from scipy.optimize import curve_fit
from natsort import natsorted
import matplotlib


tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
store_path = f"output/anderson/Plots_for_Report_20240613/T_t_analysis"
 #Save the plot in PDF format
if not os.path.exists(store_path):
      os.makedirs(store_path)
      print(f"Folder '{store_path}' created successfully.")

In [ ]:
#slices = np.array([0.7,0.9])
#slices = np.array([0,0.15,0.5,0.7,0.9])
slices = np.array([1])
#slices = np.array([0,0.15,0.3,0.5,0.7])
#nu = 0.418
nu = 0.422
#nu = 0.344
norm_factor = 4.1

In [ ]:
Db = lambda l:3e2*l/3
a=5/3
#kl = 0.9458 
kl = 0.9358
z0 = 3.25*a


#Let's calculate  adjusted values
k =  2*np.pi*(nu/(a))

l_adjusted = kl/k
Db1 = Db(l_adjusted)
adjusted_xi = 6*l_adjusted*(k*l_adjusted)**2/(1-(k*l_adjusted)**4)

In [ ]:
adjusted_xi/a

In [ ]:
#store fluxes and max times for SAL regime 

max_times = []
fluxes_SAL = []
fluxes_Diff = []

for k,path_direction in enumerate( [
                       #"RCP Sample 1 T_t slices nu 0.418/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.418 Res 30/z_incidence",
                       #"RCP Sample 1 Long Run With Monitors/z_incidence",
                       #"RCP Sample All Gap Range/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.45 Res 30/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.41 - 0.52 Res 30/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.41 - 0.52 Res 30/z_incidence"
                       #"RCP Sample 1 T_t nu 0.5 Res 30/z_incidence"
                       "RCP Sample 1 T_t nu 0.42 Res 30/z_incidence",
                       #"RCP Sample 1 T_t nu 0.42 Res 30 Abrahamh5/z_incidence"
                       #"RCP Sample 1 T_t slices nu 0.344/z_incidence"
                       #"RCP T(t) pulse 0.427 nu 18x18x18a No Structure/z_incidence"
                       ]):

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                flux_t = sim_data["time_monitorT"].flux
                flux_index = np.argmax(flux_t.values)
               
                x,y = flux_t['t'][:]*1e12,((flux_t[:]))

                item_flux = {
                        "flux":np.array(y),
                        "times":np.array(x),
                        "max_time":flux_t['t'][flux_index]*1e12,
                        "max_time_index":flux_index,
                        "size":slices[i]
                }
                if k==0:
                  
                  fluxes_SAL+=[item_flux]
                  if i==3:
                        flux_t0 = sim_data0["time_monitorT"].flux
                        x0,y0 = flux_t0['t'][:]*1e12,((flux_t0[:]/np.max(flux_t0)))
                        flux_index0 = np.argmax(flux_t0.values)
                        item_flux_0 = {
                          "flux":np.array(y0),
                          "times":np.array(x0),
                          "max_time":flux_t0['t'][flux_index0]*1e12,
                          "max_time_index":flux_index0,
                          "size":0
                          }
                        fluxes_SAL = [item_flux_0] + fluxes_SAL
                else:
                  
                  fluxes_Diff+=[item_flux]
                  if i==3:
                      flux_t0 = sim_data0["time_monitorT"].flux
                      x0,y0 = flux_t0['t'][:]*1e12,((flux_t0[:]/np.max(flux_t0)))
                      flux_index0 = np.argmax(flux_t0.values)
                      item_flux_0 = {
                        "flux":np.array(y0),
                        "times":np.array(x0),
                        "max_time":flux_t0['t'][flux_index0]*1e12,
                        "max_time_index":flux_index0,
                        "size":0
                        }
                      fluxes_Diff = [item_flux_0] + fluxes_Diff
                      
      
            

In [ ]:
#store fluxes and max times for SAL regime 

fluxes_Diff = []

#slices = np.array([0,0.15,0.5,0.7,0.9])
for k,path_direction in enumerate( [
                       #"RCP Sample 1 T_t slices nu 0.418/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.418 Res 30/z_incidence",
                       #"RCP Sample 1 Long Run With Monitors/z_incidence",
                       #"RCP Sample All Gap Range/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.45 Res 30/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.41 - 0.52 Res 30/z_incidence",
                       #"RCP Sample 1 T_t slices nu 0.41 - 0.52 Res 30/z_incidence"
                       "RCP Sample 1 T_t nu 0.5 Res 30/z_incidence"
                       #"RCP Sample 1 T_t nu 0.42 Res 30/z_incidence",
                       #"RCP Sample 1 T_t nu 0.42 Res 30 Abrahamh5/z_incidence"
                       #"RCP Sample 1 T_t slices nu 0.344/z_incidence"
                       #"RCP T(t) pulse 0.427 nu 18x18x18a No Structure/z_incidence"
                       ]):

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                flux_t = sim_data["time_monitorT"].flux
                flux_index = np.argmax(flux_t.values)
               
                x,y = flux_t['t'][:]*1e12,((flux_t[:]))

                item_flux = {
                        "flux":np.array(y),
                        "times":np.array(x),
                        "max_time":flux_t['t'][flux_index]*1e12,
                        "max_time_index":flux_index,
                        "size":slices[i]
                }
                  
                fluxes_Diff+=[item_flux]
                 
      
            

In [ ]:
font = {
        'weight' : 'bold',
        'size'   : 27}

matplotlib.rc('font', **font)



ax1 = structure_1.sim_data.simulation.sources[0].source_time.plot(times=np.linspace(0, structure_1.sim_data.simulation.run_time, 10000))

# Extract the data from the Axes object
line = ax1.get_lines()[0]
times = line.get_xdata()
amplitude = line.get_ydata()

fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(times*1e12, amplitude)
ax.set_xlabel(r"t[ps]")
ax.set_ylabel('Amplitude')
ax.set_title('Source Amplitude')
ax.legend(['Source Spectrum'])
plt.xlim(0,40)
#plt.savefig(f'{store_path}/source_time.pdf', format='pdf')
plt.show()


# Constants
c = td.C_0  # Speed of light

# Define the time array
times = np.linspace(0, structure_1.sim_data.simulation.run_time, 10000)

# Plot the spectrum and get the Axes object
ax2 = structure_1.sim_data.simulation.sources[0].source_time.plot_spectrum(times=times)

# Extract the data from the Axes object
line = ax2.get_lines()[0]
freqs = line.get_xdata()
amplitude = line.get_ydata()

# Convert frequency to wavelength (in meters)
wavelengths = c / freqs

fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(a/wavelengths, amplitude)
ax.set_xlabel(r"$\nu$'")
ax.set_ylabel('Amplitude')
ax.set_title('Source Spectrum')
ax.legend(['Source Spectrum'])
#plt.savefig(f'{store_path}/source_freqs.pdf', format='pdf')
plt.show()
# ax2 = structure_1.sim_data.simulation.sources[0].source_time.plot_spectrum(times=np.linspace(0, structure_1.sim_data.simulation.run_time, 1001))

# ax2.legend()

In [ ]:
res = structure_1.resolution
time = structure_1.run_time

(5/3)/structure_1.monitor_lambdas

In [ ]:
for i,item in enumerate(fluxes_SAL):

    max_index = fluxes_SAL[i]['max_time_index']
    print(fluxes_SAL[i]['times'][max_index])
    plt.figure(figsize=(15, 10))
    print(i)
    plt.plot(fluxes_SAL[i]['times'][:],(fluxes_SAL[i]["flux"][:]),"o",label=f"$\\nu={nu}$")
    # try:
    #     plt.plot(fluxes_Diff[i]['times'][:],fluxes_Diff[i]["flux"][:],"o",label=f"$\\nu={nu}$")
    # except:
    #     False
    plt.title(f"Raw Flux")
    plt.ylabel("Log(T(t))")
    plt.xlabel("Time[ps]")
    plt.legend()
    plt.grid()
    plt.savefig(f'{store_path}/no_structure.pdf', format='pdf')
    plt.show()
          
               








In [ ]:
#Anderson
def alpha_star(Db,xi,L):
    return (Db/xi**2)*np.exp(-L/xi)

def trans_t(t,alpha,z0,L,D,td,p=0.5,s=0.85): 
    
    # A = ((L+2*z0)**2/np.pi**2)
   

    # B = (alpha*td)**(s+1)*np.exp(-alpha*t+1)*(t*alpha)**-(p+1)
    B =(s+1)*np.log(alpha*td) + (1-alpha*td) -((p+1)*np.log(alpha*t))

    return B/np.max(B)

In [ ]:
import jax
import jax.numpy as jnp
from jaxopt import ScipyMinimize
import jaxopt
import matplotlib.pyplot as plt
jax.config.update("jax_enable_x64", True)

In [ ]:
def diffusion(t,l,L):
    diff = np.exp(-t*Db(l)*np.pi**2/(L+2*z0)**2)
    return diff/np.max(diff)

def diff(t,A):
    B = jnp.exp(-A*t)
    return jnp.log(B/jnp.max(B))

In [ ]:
for i,item in enumerate(fluxes_SAL):
    # if i < 3:
    #    continue
    max_index = fluxes_SAL[i]['max_time_index']
    plt.figure(figsize=(15, 10))
    indices = jnp.where(jnp.bool_(fluxes_SAL[i]['times']<200) & np.bool_(fluxes_SAL[i]['times']>fluxes_SAL[i]['times'][max_index]))
    plt.plot(fluxes_SAL[i]['times'][max_index:],jnp.log(fluxes_SAL[i]["flux"][max_index:]/jnp.max(fluxes_SAL[i]["flux"][max_index:])),"o",label=f"$\\nu={nu}$")
    #Theory#########################################################################
    L = 18.01*slices[i]*a
    model = lambda params,t: diff(t,params[0])
    def loss_fn(params):
     y_pred = model(params, fluxes_SAL[i]['times'][indices])
     return jnp.mean((y_pred - jnp.log(fluxes_SAL[i]["flux"][indices]/jnp.max(fluxes_SAL[i]["flux"][indices])))**2)
    
    initial_params = jnp.array([1], dtype=jnp.float32)
    optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=15000)
    # Optimize the parameters
    result = optimizer.run(initial_params)
    optimal_params = result.params

    print(optimal_params)
    transmission_diffusive = diff(fluxes_SAL[i]['times'][max_index:],optimal_params[0])
    plt.plot(fluxes_SAL[i]['times'][max_index:],transmission_diffusive,label="Diffusion")
    #################################################################################    
    plt.title(f"L={slices[i]:.3g}")
    plt.ylabel("Log(T(t))")
    plt.xlabel("Time[ps]")
    plt.legend()
    plt.grid()
    plt.show()
          
               








In [ ]:
def diffusion_jax(t,l,z0,L):
    diff = jnp.exp(-t*Db(l)*jnp.pi**2/(L+2*z0)**2)
    return jnp.log(diff/jnp.max(diff))

In [ ]:
jax.default_backend()

In [ ]:
#Anderson
Db = lambda l_var:3e2*l_var/3


def alpha_star(Db,xi,L):
    return (Db/xi**2)*jnp.exp(-L/xi)

def trans_t(t,alpha,td,p=0.5,s=0.85): 
    
    B = (alpha*td)**(s+1)*jnp.exp(-alpha*t+1)*(t*alpha)**-(p+1)
    #B =(s+1)*jnp.log(alpha*td) + (1-alpha*td) -(p+1)*jnp.log(alpha*t)
    return np.log(B/np.max(B))


def trans_t_1(t,alpha,p): 
    
    #B = (alpha*td)**(s+1)*jnp.exp(-alpha*t+1)*(t*alpha)**-(p+1)
    B = -p*jnp.log(t)-alpha*t
    #B = t**(-p)*jnp.exp(-alpha*t)

    #return jnp.log(B/jnp.max(B))
    return (B)

def transmission_anderson_func_jax(t,l,nu,L,td,s=0.85,p=0.5):
    Db_calculated = Db(l)
    k =  2*jnp.pi*(nu/(a))
    xi = 6*l*(k*l)**2/(1-(k*l)**4)
    print(xi/a)
    alpha= alpha_star(Db_calculated,xi,L)
    transmission = trans_t(t,alpha,td,p,s)
    return transmission
def transmission_anderson_func_jax_1(t,l,nu,L,td,s=0.85,p=0.5):
    Db_calculated = Db(l)
    k =  2*jnp.pi*(nu/(a))
    xi = 6*l*(k*l)**2/(1-(k*l)**4)
    alpha= alpha_star(Db_calculated,xi,L)
    transmission = trans_t_1(t,alpha,td,p,s)
    return transmission

In [ ]:

###########Fitting procedure###################
for i,item in enumerate(fluxes_SAL):
    limit = 140
    # if i ==0:
    #   continue
    # if i < 3:
    #    limit = 25
    # else: 
    #   limit = 200
    max_index = fluxes_SAL[i]['max_time_index']
    max_index_diff = fluxes_Diff[i]['max_time_index']
    td = fluxes_SAL[i]['times'][max_index]
    plt.figure(figsize=(15, 10))
    indices = jnp.where(jnp.bool_(fluxes_SAL[i]['times']<limit) & np.bool_(fluxes_SAL[i]['times']>fluxes_SAL[i]['times'][max_index]))
    indices_diff = jnp.where(jnp.bool_(fluxes_Diff[i]['times']<limit) & np.bool_(fluxes_Diff[i]['times']>fluxes_Diff[i]['times'][max_index_diff]))
    plt.plot(fluxes_SAL[i]['times'][indices],jnp.log(fluxes_SAL[i]["flux"][indices]/jnp.max(fluxes_SAL[i]["flux"][indices])),"o",label=f"$\\nu={nu}$")
    plt.plot(fluxes_Diff[0]['times'][indices_diff],jnp.log(fluxes_Diff[0]["flux"][indices_diff]/jnp.max(fluxes_Diff[0]["flux"][indices_diff])),"o",label=f"$\\nu={0.5}$")
    #Theory Anderson#########################################################################
    L = 18.01*slices[i]*a
    model = lambda params,t: trans_t_1(t,params[0],params[1])
    def loss_fn(params):
     y_pred = model(params, fluxes_SAL[i]['times'][indices])
     return jnp.mean((y_pred - jnp.log(fluxes_SAL[i]["flux"][indices]/jnp.max(fluxes_SAL[i]["flux"][indices])))**2)
    
    initial_params = jnp.array([0.5,0.5], dtype=jnp.float32)
    optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=15000)
    # Optimize the parameters
    result = optimizer.run(initial_params)
    optimal_params = result.params

    print("Anderson: ",optimal_params)
    print("Alpha Theory:",alpha_star(Db(l_adjusted),adjusted_xi,L))

    transmission_anderson = trans_t_1(fluxes_SAL[i]['times'][indices],optimal_params[0],optimal_params[1])
    theoretical_anderson = transmission_anderson_func_jax(fluxes_SAL[i]['times'][indices],l_adjusted,nu,L,td,s=0.85,p=5)

    plt.plot(fluxes_SAL[i]['times'][indices],transmission_anderson,label="Anderson Fitted")
    plt.plot(fluxes_SAL[i]['times'][indices],theoretical_anderson,label="Theoretical Anderson")
    #################################################################################    
    #Theory Diffusion#########################################################################
    model = lambda params,t: diff(t,params[0])
    def loss_fn(params):
     y_pred = model(params, fluxes_SAL[i]['times'][indices])
     return jnp.mean((y_pred - jnp.log(fluxes_SAL[i]["flux"][indices]/jnp.max(fluxes_SAL[i]["flux"][indices])))**2)
    
    initial_params = jnp.array([1], dtype=jnp.float32)
    optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=15000)
    # Optimize the parameters
    result = optimizer.run(initial_params)
    optimal_params = result.params

    print("Diff: ",optimal_params)
    transmission_diffusive = diff(fluxes_SAL[i]['times'][indices],optimal_params[0])
    plt.plot(fluxes_SAL[i]['times'][indices],transmission_diffusive,label="Diffusion Fitted")
    ###############################################################################
    plt.title(f"Transmission T(t)")
    plt.ylabel("Log(T(t))")
    plt.xlabel("Time[ps]")
    plt.legend()
    plt.grid()
    plt.savefig(f'{store_path}/{f"L_{slices[i]:.3g}_{nu}_res_{res}_time_{time}"}.pdf', format='pdf')
    plt.show()
          
               






